**Install**:
```
pip install -U langchain \
    langgraph==0.1.19 \
    langchain_openai \
    langchainhub \
    langchain_experimental
```

In [1]:
import os
from dotenv import load_dotenv
load_dotenv('./.env')

os.environ["LANGCHAIN_PROJECT"] = "Demo LangGraph 001"

### GraphState

In [2]:
import operator
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    agent_outcome: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

### Custom Tools

In [3]:
from langchain.tools import BaseTool, StructuredTool, Tool, tool
import random

@tool("lower_case", return_direct=True)
def to_lower_case(input:str) -> str:
    """Retorna a entrada em minúsculas"""
    return input.lower()

@tool("random_number", return_direct=True)
def random_number_maker(input:str) -> str:
    """Retorna um número entre 0 e 100"""
    return random.randint(0, 100)

tools = [to_lower_case, random_number_maker]

In [4]:
random_number_maker.run('random')

22

In [5]:
to_lower_case.run('DANIEL')

'daniel'

### Agent com (create_openai_functions_agent)

In [6]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI

prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", streaming=True)

agent_runnable = create_openai_functions_agent(llm, tools, prompt)

In [7]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplat

In [8]:
prompt.get_prompts()

[ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTempla

In [9]:
inputs = {"input": "gere um número aleatório e escreva por extenso em minúsculas.",
          "chat_history":[],
          "intermediate_steps":[]}

agent_outcome = agent_runnable.invoke(inputs)

In [10]:
agent_outcome

AgentActionMessageLog(tool='random_number', tool_input={'input': 'random'}, log="\nInvoking: `random_number` with `{'input': 'random'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"random"}', 'name': 'random_number'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-57b3baa7-1dbf-4018-8033-4b90e4b4d445-0')])

In [11]:
type(agent_runnable)

langchain_core.runnables.base.RunnableSequence

### Nodes

In [12]:
from langchain_core.agents import AgentFinish
from langgraph.prebuilt.tool_executor import ToolExecutor

tool_executor = ToolExecutor(tools)

In [13]:
def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}

def execute_tools(data):
    agent_action = data['agent_outcome']
    output = tool_executor.invoke(agent_action)
    print(f"The agent action is {agent_action}")
    print(f"The tool result is: {output}")
    return {"intermediate_steps": [(agent_action, str(output))]}

def should_continue(data):
    if isinstance(data['agent_outcome'], AgentFinish):
        return "end"
    else:
        return "continue"

### Graph

In [14]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(AgentState)

workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)
workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END
    }
)

workflow.add_edge('action', 'agent')

app = workflow.compile()

In [15]:
workflow.branches

defaultdict(dict,
            {'agent': {'should_continue': Branch(path=should_continue(recurse=True), ends={'continue': 'action', 'end': '__end__'}, then=None)}})

In [16]:
workflow.nodes, workflow.edges

({'agent': StateNodeSpec(runnable=agent(recurse=True), metadata=None, input=<class '__main__.AgentState'>, retry_policy=None),
  'action': StateNodeSpec(runnable=action(recurse=True), metadata=None, input=<class '__main__.AgentState'>, retry_policy=None)},
 {('__start__', 'agent'), ('action', 'agent')})

In [17]:
workflow.channels

{'input': <langgraph.channels.last_value.LastValue at 0x116831bb0>,
 'chat_history': <langgraph.channels.last_value.LastValue at 0x116831c10>,
 'agent_outcome': <langgraph.channels.last_value.LastValue at 0x116831be0>,
 'intermediate_steps': <langgraph.channels.binop.BinaryOperatorAggregate at 0x116831e50>}

In [18]:
inputs = {"input": "gere um número aleatório e escreva por extenso em minúsculas.", "chat_history": []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'agent_outcome': AgentActionMessageLog(tool='random_number', tool_input={'input': 'random'}, log="\nInvoking: `random_number` with `{'input': 'random'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"random"}', 'name': 'random_number'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-8c32735e-a663-4ca0-be60-06017983b8ae-0')])}
----
The agent action is tool='random_number' tool_input={'input': 'random'} log="\nInvoking: `random_number` with `{'input': 'random'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"random"}', 'name': 'random_number'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-8c32735e-a663-4ca0-be60-06017983b8ae-0')]
The tool result is: 54
{'intermediate_steps': [(AgentActionMessageLog(tool='random_number', tool_input={'input': 'random'}, log="\nInvoking: `ra

In [19]:
inputs = {"input": "gere um número aleatório e escreva por extenso em minúsculas.", "chat_history": []}
output = app.invoke(inputs)

The agent action is tool='random_number' tool_input={'input': 'random'} log="\nInvoking: `random_number` with `{'input': 'random'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"random"}', 'name': 'random_number'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-35d31fac-cc7a-468c-8b2f-1f046bf9ce8d-0')]
The tool result is: 92
The agent action is tool='lower_case' tool_input={'input': 'noventa e dois'} log="\nInvoking: `lower_case` with `{'input': 'noventa e dois'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"noventa e dois"}', 'name': 'lower_case'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-c497d890-5175-43f4-b46f-afb6a19c8122-0')]
The tool result is: noventa e dois


In [20]:
output.get("agent_outcome").return_values['output']

'O número aleatório gerado foi 92, e por extenso em minúsculas é "noventa e dois".'

In [21]:
output.get("intermediate_steps")

[(AgentActionMessageLog(tool='random_number', tool_input={'input': 'random'}, log="\nInvoking: `random_number` with `{'input': 'random'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"random"}', 'name': 'random_number'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-35d31fac-cc7a-468c-8b2f-1f046bf9ce8d-0')]),
  '92'),
 (AgentActionMessageLog(tool='lower_case', tool_input={'input': 'noventa e dois'}, log="\nInvoking: `lower_case` with `{'input': 'noventa e dois'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"noventa e dois"}', 'name': 'lower_case'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-c497d890-5175-43f4-b46f-afb6a19c8122-0')]),
  'noventa e dois')]

In [22]:
inputs = {"input": "quantos meses tem 1 ano?", "chat_history": []}

output = app.invoke(inputs)

In [23]:
output.get("agent_outcome").return_values['output']

'Um ano tem 12 meses.'

In [24]:
output.get("intermediate_steps")

[]